1.6.1: Functions as Arguments

In [20]:
def summation(n, func):
    k, res = 1, 0
    while k <= n:
        res, k = res + func(k), k + 1
    return res

def cube(x):
    return x*x*x

def identity(x):
    return x

def pi_term(x):
    return 8/((4 * x - 3) * (4 * x - 1))

def sum_cubes(n):
    return summation(n, cube)

def sum_naturals(n):
    return summation(n, identity)

def pi_sum(n):
    return summation(n, pi_term)

In [21]:
sum_cubes(10)

3025

In [22]:
sum_naturals(10)

55

In [23]:
pi_sum(1000)

3.141092653621038

1.6.2 Functions as General Methods

In [24]:
def improve(close, update, guess = 1):
    while not close(guess):
        guess = update(guess)
    return guess

In [25]:
def golden_update(guess):
    return 1/guess + 1

def golden_close(guess):
    return approx_eq(guess * guess, guess + 1)

def approx_eq(x, y, threshold = 1e-15):
    return abs(x - y) < threshold

In [26]:
improve(golden_close, golden_update, -100000000)

1.6180339887498951

1.6.3: Defining Functions III: Nested Definitions

In [27]:
def sqrt(a):
    def sqrt_update(x):
        return (x + a/x) / 2
    def sqrt_close(x):
        return approx_eq(x * x, a)
    return improve(sqrt_close, sqrt_update)

In [28]:
sqrt(9)

3.0

1.6.4: Functions as Returned Values

In [ ]:
def compose(f, g):
    def h(x):
        return f(g(x))
    return h